In [ ]:
import os
import json
import math
import matplotlib.pyplot as plt

# Set angle unit: 'radians' or 'degrees'
angle_unit = 'radians'

def quaternion_to_yaw(q):
    """
    Convert a quaternion [qx, qy, qz, qw] into a yaw angle (rotation about the up-axis).
    Assumes the horizontal plane is x-z and y is vertical.
    """
    qx, qy, qz, qw = q
    # yaw = math.atan2(2 * (qw * qy + qx * qz), 1 - 2 * (qy * qy + qz * qz))
    yaw = qy
    # Convert to degrees if necessary
    if angle_unit == 'degrees':
        yaw = math.degrees(yaw)
    return yaw

def load_points(file_path, point_type):
    """
    Loads point data from a JSON file and tags each point with its type.
    Expects a JSON list where each element is a dictionary containing a 'name' and a 'transform'
    with 'translation' (a list of three numbers) and 'rotation' (a quaternion as a list of four numbers).
    """
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return []

    points = []
    for item in data:
        name = item.get("name", "")
        translation = item.get("transform", {}).get("translation", [0, 0, 0])
        rotation = item.get("transform", {}).get("rotation", [0, 0, 0, 1])
        # Use x and z for the 2D representation (assuming y is up)
        x, _, z = translation
        yaw = quaternion_to_yaw(rotation)
        # Tag the point with its type ("blocker" or "shot")
        points.append({"name": name, "x": x, "z": z, "yaw": yaw, "type": point_type})
    return points


def plot_map(points, output_filename, scene_name):
    """
    Create a 2D plot of points with arrows showing direction and text labels.
    Blockers and Shots are colored differently.
    The plot is saved as a PDF with the given output_filename.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Define color mapping for point types:
    colors = {
        "blocker": {"dot": "black", "arrow": "black", "label": "black"},
        "shot": {"dot": "red", "arrow": "red", "label": "red"}
    }
    
    #We could check for blockers named the same and then draw lines between them

    for p in points:
        x = p["x"]
        z = p["z"]
        yaw = p["yaw"]
        p_type = p.get("type", "shot")
        color = colors.get(p_type, colors["shot"])
        
        if angle_unit == 'degrees':
            yaw_rad = math.radians(yaw)
        else:
            yaw_rad = yaw

        # Plot the point using the type-specific shape color.
        ax.plot(x, z, 'o', color=color["dot"], marker="")
        # Set an arrow length for indicating direction.
        arrow_length = 0.2
        
        # Convert yaw from degrees to radians for plotting if necessary

        dx = arrow_length * math.sin(yaw_rad)
        dz = arrow_length * math.cos(yaw_rad)
        ax.arrow(x, z, dx, dz, head_width=0.05, head_length=0.05, fc=color["arrow"], ec=color["arrow"])
        ax.text(x, z, f' {p["name"]}', fontsize=9, color=color["label"])
        
    # ax.set_xlabel("X")
    # ax.set_ylabel("Z")
    ax.set_title(f"Map for {scene_name.replace("_", " ")}")
    ax.grid(False)
    ax.axis('equal')
    ax.set_axis_off()
    plt.savefig(output_filename)
    plt.close(fig)
    print(f"Map saved to {output_filename}")

# Find folders in the current directory that start with 'Scene_'
scene_folders = [folder for folder in os.listdir('.') if os.path.isdir(folder) and folder.startswith("Scene_")]

if not scene_folders:
    print("No scene folders found.")

for scene in scene_folders:
    print(f"Processing folder: {scene}")
    scene_path = os.path.join(".", scene)
    
    # Construct paths for JSON files in the scene folder.
    blocker_file = os.path.join(scene_path, "Blocker.json")
    shotframe_file = os.path.join(".", "Scene_1ShotFrame.json")
    
    points = []
    # Load points from Blocker.json with type 'blocker' if it exists.
    if os.path.exists(blocker_file):
        points.extend(load_points(blocker_file, "blocker"))
    else:
        print(f"File not found: {blocker_file}")
    
    # Load points from ShotFrame.json with type 'shot' if it exists.
    if os.path.exists(shotframe_file):
        points.extend(load_points(shotframe_file, "shot"))
    else:
        print(f"File not found: {shotframe_file}")
    
    # Only generate a map if there are any points.
    if points:
        output_filename = f"{scene}_Map.pdf"
        plot_map(points, output_filename, scene)
    else:
        print(f"No points found in {scene}.")


Processing folder: Scene_0
Map saved to Scene_0_Map.pdf
Processing folder: Scene_1
File not found: ./Scene_1/Blocker.json
Map saved to Scene_1_Map.pdf


/var/folders/2z/2jz4yqyd06v2fxf77_y7d8rm0000gn/T/ipykernel_68418/1592882130.py:77: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  ax.plot(x, z, 'o', color=color["dot"], marker="")
/var/folders/2z/2jz4yqyd06v2fxf77_y7d8rm0000gn/T/ipykernel_68418/1592882130.py:77: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  ax.plot(x, z, 'o', color=color["dot"], marker="")
